### Front end implementation on a subset of raw data (Done during preprocessing & computation of more advanced model)

In [ ]:
import streamlit as st
import pandas as pd
import pickle
import numpy as np

# Loading the beer data
beers = pickle.load(open('beer_list.pkl', 'rb'))
df = pd.read_pickle('popular_df.pkl')
rat_similarity = pickle.load(open('rat_sim.pkl', 'rb'))

#Content based Recommendation Function
def recommend(selected_beer):
    index = beers[beers['beer/name'] == selected_beer].index
    if not index.empty:
        index = index[0]
        similarity_values = rat_similarity[index]
        indices_descending = np.argsort(similarity_values)[::-1]
        #Recommending the top 5 beers according to the selected beer by user
        top_indices = [i for i in indices_descending if i != index][:5]
        recommended_beer_names = [beers.iloc[i]['beer/name'] for i in top_indices]
        return recommended_beer_names
    else:
        return []

# Function to get recommendations of popular beers only
def popular_recommend(beer_name):
    index = np.where(df['beer/name'] == beer_name)[0]
    if len(index) == 0:
        return []
    similar_items = sorted(list(enumerate(rat_similarity[index[0]])), key=lambda x: x[1], reverse=True)[1:6]
    recommendations = []
    for i in similar_items:
        temp_df = df[df['beer/name'] == df.iloc[i[0]]['beer/name']]
        recommendations.append((temp_df.iloc[0]['beer/name'], temp_df.iloc[0]['review/overall'], temp_df.iloc[0]['beer/style']))
    return recommendations

background_image_url = 'https://images.unsplash.com/photo-1600788886242-5c96aabe3757?q=80&w=1887&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D'
st.image(background_image_url, width=106)
st.title("Beer Recommendation System")


# Input Section for user to select a beer
selected_beer = st.selectbox("Type or select a beer from the dropdown", beers['beer/name'].values)
#Showing the Alcohol By Volume of the Selected Beer
abv = beers.loc[beers['beer/name'] == selected_beer, 'beer/ABV'].values
if len(abv) > 0:
    st.write(f"**ABV (Alcohol By Volume) of {selected_beer}:** {abv[0]}")
if selected_beer:
    # st.subheader(f"Visualizations for {selected_beer}")

    # Filtering dataframe to get ratings for selected beer
    selected_beer_ratings = beers[beers['beer/name'] == selected_beer]['review/overall'].values

    # Creating a histogram of ratings
    # st.bar_chart(selected_beer_ratings)

# Giving recommendations by textual review
if st.button("Get Recommendations by Review"):
    recommendations = recommend(selected_beer)
    if recommendations:
        st.write("Top 5 Recommended Beers:")
        for recommendation in recommendations:
            st.write(recommendation)
    else:
        st.write("No recommendations found for the selected beer.")

# Giving dropdown to User for selecting a beer for popularity-based recommendations
selected_popular_beer = st.selectbox("Type or select a beer from the dropdown", df['beer/name'].values)

# Giving recommendations by popularity
if st.button("Get Recommendations by Popularity"):
    recommendations = popular_recommend(selected_popular_beer)
    if recommendations:
        st.write("Top 5 Recommended Beers:")
        recommendations_df = pd.DataFrame(recommendations, columns=['Beer Name', 'Overall Rating', 'Beer Style'])
        st.dataframe(recommendations_df)
    else:
        st.write("No recommendations found for the selected beer.")
